# Creating your own ERC-20 Token

- statically typed language

### Types 

* Boolean
* Numbers
* String (which is just a specical type of array)
* Array
* Address
* Contract
* Enum
* Mapping
* Function
* Custom

![solidity types](solidity-types.png)

### Numbers

- most common = unsigned integer (uint) = positive integer 0+

![uint](uint-types.png)

- probably easier to just default for uint or uint256 (this is the same thing)

- int = a signed integer (e.g. you can have negative numbers as well)

![int types](int-types.png)

### Constant & Immutable

- using CONSTANT or IMMUTABLE on variables that you know will not change, can save you gas! 

- there is a difference tho

- CONSTANT needs to be declared at compile time
- IMMUTABLE can be declared once when contract is created

- good practice to use upper-case to easily spot them

![constant-immutable](constant-immutable.png)

## ERC vs EIP 

- ERC = Ethereum Request for Comment
- EIP = Ethereum Improvement Proposal 

* [Ethereum EPIs](https://eips.ethereum.org/)
* [EIP GitHub Repo](https://github.com/ethereum/EIPs)
* [Ethereum ERCs](https://eips.ethereum.org/erc)

- anyone can make a proposal to improve Ethereum, just need to make a pull request on EIP Repo

- Dev's most likely to deal with ERCs

- ERCs = standards

## ERC20

- [EIP for ERC20](https://eips.ethereum.org/EIPS/eip-20)
- most popular ERC by fange
- this is a Fungible Token
- this means, every token is replacable with any other token of the same kind 
- essentially 1Ether is the same as any other 1Ether

![erc-20-func](erc20-common-func.png)
---
- First 3 functions = token
- Next 3 functions = smartContract (important - do not send tokens to contract = many new dev's lost their money this way)
- Final 3 functions - optional - mostly in Tokens, but are optional 
---
### vs NFT
- NFT = no two are the same
- all unique 

**Setting up the environment**
==============================

Let's set up your dev environment! You will need Node.js and we will also be using **Hardhat** as our development framework.

To do this you'll need to do the following:

1.  Install Node.js 12.0 or greater (or upgrade your Node.js installation to 12.0 or greater)
2.  Install a code editor (if you don't already have a code editor)
3.  Install the Solidity extension

Below I explain how to do each of these.



**1\. Install Node.js**
-----------------------

Most Ethereum libraries and tools are written in JavaScript, and so is Hardhat. If you're not familiar with Node.js, it's a JavaScript runtime built on Chrome's V8 JavaScript engine. It's the most popular solution to run JavaScript outside of a web browser and Hardhat is built on top of it.

You can skip this section if you already have a working Node.js `>=12.0` installation. If not, here's how to install it on Ubuntu, MacOS and Windows.

### Linux/Ubuntu

Make sure you have `git` installed. Otherwise, follow [these instructions](https://www.atlassian.com/git/tutorials/install-git).

Copy and paste these commands in a terminal:

sudo apt update
sudo apt install curl git
curl -sL https://deb.nodesource.com/setup\_12.x | sudo -E bash -
sudo apt install nodejs

### MacOS

Make sure you have `git` installed. Otherwise, follow [these instructions](https://www.atlassian.com/git/tutorials/install-git).

There are multiple ways of installing Node.js on MacOS. We will be using [Node Version Manager (nvm)](http://github.com/creationix/nvm). Copy and paste these commands in a terminal:

curl -o- https://raw.githubusercontent.com/creationix/nvm/v0.35.2/install.sh | bash
nvm install 12
nvm use 12
nvm alias default 12
npm install npm --global # Upgrade npm to the latest version

### Windows

Installing Node.js on Windows requires a few manual steps. Download and run these:

1.  First install Git from [here](https://github.com/git-for-windows/git/releases/latest).
2.  Download node-v12.XX.XX-x64.msi file from [here](https://nodejs.org/dist/latest-v12.x/ ).

If you don't have a terminal to enter commands yet, I recommend using Git Bash from [here](https://gitforwindows.org/). This will give you a similar terminal to those with Mac or Linux.

  

**2\. Install Visual Studio Code**
----------------------------------

If you do not already have a code editor installed, then download and install VS Code from [here](https://code.visualstudio.com/download).

  

**3\. Install Solidity extension**
----------------------------------

Install the Solidity extension either [here](https://marketplace.visualstudio.com/items?itemName=JuanBlanco.solidity) or [here](https://marketplace.visualstudio.com/items?itemName=ylrednet.ytidilos).

You can see a list of alternative development frameworks over at [https://ethereum.org/en/developers/local-environment/](https://ethereum.org/en/developers/local-environment). 

  

### Getting it running
The below code can be used to run the project setup, and should be used in case the commands you ran from the course caused any issues.

```
mkdir my-project
cd my-project
npm init --yes                              // creates JSON
npm install --save-dev hardhat              // downloads hardhat
npm install --save-dev 

npx hardhat
> Choose: Create an advanced sample project that uses TypeScript
```

## Memory / Storage / Calldata

### 3 keywords
---
- *storage* - state variable, stored on blockchain
- *memory* - variable is in memory, exists only while function is being called
- *calldata* - special data location, like memory but cheaper

# Global variable msg

- msg.sender:       sender of the message
- msg.value:        number of ether sent (in Wei)
- msg.data:         calldata
- msg.sig:          first four bytes of calldata


we use *Sender* and *Value* most often

msg.data - usually not called directly, they're essentially the parameters within a function 

msg.sig - how are the 4 bytes calculate? 
- function name
- function input parameter types
- hash256 of this
- solidity does this itself

 as such, we rarely need to access msg.sig ourselves
---
most notable is that there can be collisions with this 4 byte calculation and there have been attacks targetting this, so just something to be aware of. 


# Address

- 20 bytes long 
- can be Normal or Payable
- basically the same, but PAYABLE can receive Ether 

### Example of transfering 

In [ ]:
// vulnerable
payMeBackLess(msg.sender).transfer(amount);

// is the same as (but this is safer)
(bool, success,) = payable(msg.sender).call{gas:2300,value:amount}(""); // limits how much gas can be used
require(success, "Transfer Failed.");

// better
(bool, success,) = payable(msg.sender).call{ value: amount}("");        // better to leave gas field empty
require(success, "Transfer failed.");                                   // gas will instead use any gas left in the contract

below is pretty much the entire required syntax for an ERC20 token

In [ ]:
// SPDIX-Licence-Identification: MIT
pragma solidity 0.8.11;


contract ERC20 {
    // declare our variables (this is a strongly typed language remember)
    uint256 public totalSupply;
    string public name;
    string public symbol;
    // uint public decimals;

    // we need some where to map our balences
    mapping(address => uint256) public balanceOf; 
    mapping(address => mapping(address => uint256)) public allowance;

    constructor(string memory _name, string memory _symbol) {
        name = _name;
        symbol = _symbol;
    }

    function decimals() external pure returns (uint8) {     // declaring it here is cheaper from gas PoV
        return 18;
    }

    function transfer(address recipient, uint256 amount) external returns (bool) {
        require(recipient != address(0), 'ERC20: transfer to sero address');
        
        uint256 senderBalance = balanceOf[msg.sender];

        require(senderBalance >= amount, 'ERC20: transfer amount exceed balance');

        balanceOf[msg.sender] = senderBalance - amount;
        balanceOf[recipient] += amount;

        return true;
    }

        function decimals() external pure returns (uint8) {     // declaring it here is cheaper from gas PoV
        return 18;
    }

    function transfer(address recipient, uint256 amount) external returns (bool) {
        return _transfer((msg.sender, recipient, amount);
    }

    function transferFrom(address sender, address recipient, uint256 amount) external returns (bool) {
        uint256 currentAllowance = allowance[sender][msg.sender];

        require(
            currentAllowance >= amount,
            "ERC20: transfer not allowed"
        );

        allowance[sender][msg.sender] = currentAllowance - amount;
        
        return _transfer((sender, recipient, amount);
    }

    function approve(address sender, uint256 amount) external returns(bool) {
        require(sender != address(0), 'ERC20: approve to the zero address');

        allowance[msg.sender][spender] = amount;

        return true;
    }

    function _transferAddress(address recipient, uint256 amount) private returns (bool) {
        require(recipient != address(0), 'ERC20: transfer to sero address');
        
        uint256 senderBalance = balanceOf[sender];

        require(senderBalance >= amount, 'ERC20: transfer amount exceed balance');

        balanceOf[sender] = senderBalance - amount;
        balanceOf[recipient] += amount;

        return true;
    }
}

this can technically be deployed to the Ethereum blockchain now, though it is still missing events, so we will look at those next: 

first we might want to test the contract. We could consider uploading it to a testNet and engaging the contract that way. but this takes a lot of time, even testing small things can take a lot of time. 
---
## Automated Testing

Interestingly, when you consider Automated testing vs the code required for a typical smartContract - you may actually produce more code for the testing than you do for the smart contract 

### Unit Testing
- test small units of the code at a time
### Integration Testing
- test the code all at once 
---
To test, we want a blockchain that runs on our computer. 
- use a simple library like the JavaScript library; **Mocha**

![mocha](mocha-js-library.png)

![mocha-2](mocha-example.png)

- we describe tests for different scenarios

![tests-ex](testing-example.png)


We can break that down: 

![test-1](test-1-deploy.png)

- we're saying that each time the contract is deployed
- we expect positive

![test-2](test-2-describe-amount.png)

- we are going to mint a defined number of coins for this test
- we expect a positive result

![test-3](test-3-transfer-ten.png)

- we want to test what happens when we transfer an amount of coins that we know we have
- we also want to verify that the transfer happened
- we expect a positive result 

![test-4](test-4-transfer-too-many.png)

- we want to know what happens when we try to transfer more than we own
- we expect this to fail (but in a known way) 
- expect negative
